<a href="https://colab.research.google.com/github/Areej-Ah/Library-Ontology/blob/main/MWS_WSM_S24_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 7.3 MB/s eta 0:00:00


In [ ]:
import rdflib

In [ ]:
g = rdflib.Graph()
g.parse("Library.rdf", format="xml")

<Graph identifier=N46f2997860fd49b9890d191c1f58cf15 (<class 'rdflib.graph.Graph'>)>

In [ ]:
print(f"Number of triples in the graph: {len(g)}")

Number of triples in the graph: 194


In [ ]:
query1 = """
PREFIX : <http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

SELECT ?book (AVG(?rating) AS ?avgRating) ?year
WHERE {
    ?book rdf:type :Book .
    ?book :publishedInYear ?year .
    ?book :hasReview ?review .
    ?review :hasRating ?rating .

    FILTER(?year >= 2000 && ?year <= 2020)
}
GROUP BY ?book ?year
ORDER BY DESC(?avgRating)
"""

results1 = g.query(query1)
print("Books in year range(2000-2020), ordered by average rating:")
for row in results1:
    print(f"Book: {row.book}, Year: {row.year}, Average Rating: {row.avgRating}")

Books in year range(2000-2020), ordered by average rating:
Book: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Book1, Year: 2002, Average Rating: 4.5
Book: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Book3, Year: 2014, Average Rating: 4.0
Book: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Book2, Year: 2012, Average Rating: 3.35


In [ ]:
query2 = """
PREFIX : <http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

SELECT ?author (SUM(?rating) AS ?totalRating)
WHERE {
    ?book rdf:type :Book .
    ?book :writtenBy ?author .
    ?book :hasReview ?review .
    ?review :hasRating ?rating .
}
GROUP BY ?author
ORDER BY DESC(?totalRating)
"""

results2 = g.query(query2)
print("Author(s) with the highest total review ratings:")
for row in results2:
    print(f"Author: {row.author}, Total Rating: {row.totalRating}")

Author(s) with the highest total review ratings:
Author: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Author1, Total Rating: 17.5
Author: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Author2, Total Rating: 6.7


In [ ]:
query3 = """
PREFIX : <http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

SELECT ?publisher (AVG(?rating) AS ?avgRating)
WHERE {
    ?book rdf:type :Book .
    ?book :publishedBy ?publisher .
    ?book :hasReview ?review .
    ?review :hasRating ?rating .
}
GROUP BY ?publisher
ORDER BY DESC(?avgRating)
"""

results3 = g.query(query3)
print("Publishers and their average book ratings, sorted by rating:")
for row in results3:
    print(f"Publisher: {row.publisher}, Average Rating: {row.avgRating}")

Publishers and their average book ratings, sorted by rating:
Publisher: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Publisher1, Average Rating: 4.5
Publisher: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Publisher3, Average Rating: 4.0
Publisher: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Publisher2, Average Rating: 3.35


In [ ]:
query_genres = """
PREFIX : <http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

SELECT ?genre (AVG(?rating) AS ?avgRating)
WHERE {
    ?book rdf:type :Book .
    ?book :belongsToGenre ?genre .
    ?book :hasReview ?review .
    ?review :hasRating ?rating .
}
GROUP BY ?genre
ORDER BY DESC(?avgRating)
"""

results_genres = g.query(query_genres)

for row in results_genres:
    print(f"Genre: {row.genre}, Average Rating: {row.avgRating}")

Genre: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Genre1-1, Average Rating: 4.5
Genre: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Genre3-1, Average Rating: 4.0
Genre: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Genre2-1, Average Rating: 3.35


In [ ]:
query_authors_by_nationality = """
PREFIX : <http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

SELECT ?author ?nationality (AVG(?rating) AS ?avgRating)
WHERE {
    ?book rdf:type :Book .
    ?book :writtenBy ?author .
    ?author :hasNationality ?nationality .

    # Optional part for rating to include authors with no ratings
    OPTIONAL {
        ?book :hasReview ?review .
        ?review :hasRating ?rating .
    }
}
GROUP BY ?author ?nationality
ORDER BY DESC(?avgRating)

"""

results_authors_by_nationality = g.query(query_authors_by_nationality)

for row in results_authors_by_nationality:
    print(f"Author: {row.author}, Nationality: {row.nationality}, Average Rating: {row.avgRating}")

Author: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Author1, Nationality: British, Average Rating: 4.375
Author: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Author2, Nationality: British, Average Rating: 3.35
Author: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Author3, Nationality: British, Average Rating: 0


In [ ]:
query_prolific_authors = """
PREFIX : <http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

SELECT ?decade ?author (COUNT(?book) AS ?bookCount)
WHERE {
    ?book rdf:type :Book .
    ?book :writtenBy ?author .             # Linking book to author
    ?book :publishedInYear ?year .         # Linking book to publication year
    FILTER(?year >= 2000)                   # Filtering books published after 2000
    BIND(FLOOR(?year / 10) * 10 AS ?decade)  # Grouping by decade (for example: 2000s, 2000s)
}
GROUP BY ?decade ?author
ORDER BY DESC(?bookCount)
"""

results_prolific_authors = g.query(query_prolific_authors)

for row in results_prolific_authors:
    print(f"Decade: {row.decade}, Author: {row.author}, Books Published: {row.bookCount}")

Decade: 2000, Author: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Author1, Books Published: 2
Decade: 2010, Author: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Author2, Books Published: 2
Decade: 2010, Author: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Author1, Books Published: 1
Decade: 2000, Author: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Author3, Books Published: 1


In [ ]:
query_highest_rated_book = """
PREFIX : <http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

SELECT ?book ?author ?genre ?rating
WHERE {
    ?book rdf:type :Book .
    ?book :writtenBy ?author .
    ?book :belongsToGenre ?genre .
    ?book :hasReview ?review .
    ?review :hasRating ?rating .
}
ORDER BY DESC(?rating)
LIMIT 1
"""

results_highest_rated_book = g.query(query_highest_rated_book)

for row in results_highest_rated_book:
    print(f"Book: {row.book}, Author: {row.author}, Genre: {row.genre}, Rating: {row.rating}")

Book: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Book1, Author: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Author1, Genre: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Genre1-1, Rating: 4.9


In [ ]:
query_avg_rating_per_decade = """
PREFIX : <http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

SELECT ?decade (AVG(?rating) AS ?avgRating)
WHERE {
    ?book rdf:type :Book .
    ?book :publishedInYear ?year .
    ?book :hasReview ?review .
    ?review :hasRating ?rating .
    FILTER(?year >= 2000)                    # Filter for books published from 2000 onward
    BIND(FLOOR(?year / 10) * 10 AS ?decade)   # Group by decade
}
GROUP BY ?decade
ORDER BY ?decade
"""

results_avg_rating_per_decade = g.query(query_avg_rating_per_decade)

for row in results_avg_rating_per_decade:
    print(f"Decade: {row.decade}, Average Rating: {row.avgRating}")

Decade: 2000, Average Rating: 4.5
Decade: 2010, Average Rating: 3.566666666666666666666666667


In [ ]:
query_highest_rated_book_per_genre = """
PREFIX : <http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

SELECT ?genre ?book (MAX(?rating) AS ?maxRating)
WHERE {
    ?book rdf:type :Book .
    ?book :belongsToGenre ?genre .
    ?book :hasReview ?review .
    ?review :hasRating ?rating .
}
GROUP BY ?genre
ORDER BY ?genre
"""

results_highest_rated_book_per_genre = g.query(query_highest_rated_book_per_genre)

for row in results_highest_rated_book_per_genre:
    print(f"Genre: {row.genre}, Book: {row.book}, Highest Rating: {row.maxRating}")

Genre: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Genre1-1, Book: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Book1, Highest Rating: 4.9
Genre: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Genre2-1, Book: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Book2, Highest Rating: 3.5
Genre: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Genre3-1, Book: http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Book3, Highest Rating: 4.0


In [ ]:
!pip install owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 38.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.47-cp310-cp310-linux_x86_64.whl size=24075202 sha256=06df40fe7a89e2cc16a43bb30479bcce6a10d8a655edf321448b0436e6b8d8a2
  Stored in directory: /root/.cache/pip/wheels/27/3e/ba/4171c4b10bba9fe1774fbf8fcf794de889e636ce64ad83a533
Successfully built owlready2


In [ ]:
from owlready2 import *

In [ ]:
# Load ontology
onto = get_ontology("Library.rdf").load()

# Run reasoner
with onto:
    sync_reasoner()

# Check for instances with inconsistencies
for book in onto.Book.instances():
    genres = list(book.belongsToGenre)
    if len(genres) > 1:
        print(f"Book '{book}' belongs to multiple genres: {[genre.name for genre in genres]}")

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/lib/python3.10/dist-packages/owlready2/hermit:/usr/local/lib/python3.10/dist-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmpxj0snez6
* Owlready2 * HermiT took 2.6505982875823975 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [ ]:
from rdflib import Namespace, URIRef

NS = Namespace("http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#")
g.bind("ns", NS)

In [ ]:
def recommend_books_by_genre(user_id):
    # Query the user's preferred genres
    preferred_genre_query = """
    PREFIX ns: <http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

    SELECT ?genre
    WHERE {
        ?user ns:hasUserID """ + str(user_id) + """ .  # Ensure user_id is properly inserted
        ?user ns:preferredGenre ?genre .
    }
    """
    preferred_genres = [str(row[0]) for row in g.query(preferred_genre_query)]

    print("Preferred genres:", preferred_genres)

    # Query books in the preferred genres
    book_query = """
    PREFIX ns:<http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

    SELECT ?book ?genre
    WHERE {
        ?book ns:belongsToGenre ?genre .
        FILTER (?genre IN (""" + ", ".join(f"<{genre}>" for genre in preferred_genres) + """))
    }
    """
    recommended_books = g.query(book_query)
    return [(str(book[0]), str(book[1])) for book in recommended_books]

In [ ]:
def recommend_highly_rated_books(user_id, rating_threshold=4.0):
    # Convert user_id to string in the query
    read_books_query = """
    PREFIX ns:<http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

    SELECT ?book
    WHERE {
        ?user ns:hasUserID """ + str(user_id) + """ .
        ?user ns:hasRead ?book .
    }
    """
    read_books = {str(row[0]) for row in g.query(read_books_query)}

    # Query for highly-rated books the user hasn't read
    book_query = """
    PREFIX ns:<http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

    SELECT ?book ?rating
    WHERE {
        ?book ns:hasReview ?review .
        ?review ns:hasRating ?rating .
        FILTER (?rating >= """ + str(rating_threshold) + """)
        FILTER (?book NOT IN (""" + ", ".join(f"<{book}>" for book in read_books) + """))
    }
    """
    recommended_books = g.query(book_query)
    return [(str(book[0]), float(book[1])) for book in recommended_books]

In [ ]:
def recommend_books_based_on_reviews(user_id, review_threshold=4.0):
    # Convert user_id to string in the query
    # Query to fetch books the user has read
    read_books_query = """
    PREFIX ns:<http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

    SELECT ?book
    WHERE {
        ?user ns:hasUserID """ + str(user_id) + """ .
        ?user ns:hasRead ?book .  # Books the user has read
    }
    """
    read_books = {str(row[0]) for row in g.query(read_books_query)}
    print("Books that user has read:", read_books)

    # If no books are read, return early
    if not read_books:
        print("No books found that the user has read.")
        return []

    # Query to fetch all books
    all_books_query = """
    PREFIX ns:<http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

    SELECT ?book
    WHERE {
        ?book a ns:Book .
    }
    """
    all_books = {str(row[0]) for row in g.query(all_books_query)}
    print("All books in the library:", all_books)

    # Find books that the user has NOT read
    unread_books = all_books - read_books
    print("Unread books for user:", unread_books)

    # Query books with high reviews by other users
    review_based_query = """
    PREFIX ns:<http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#>

    SELECT ?book ?review_rating
    WHERE {
        ?book ns:hasReview ?review .
        ?review ns:hasRating ?review_rating .
        FILTER (?review_rating >= """ + str(review_threshold) + """)
        FILTER (?book IN (""" + ", ".join(f"<{book}>" for book in unread_books) + """))
    }
    """
    reviewed_books = g.query(review_based_query)

    # Check if there are any reviewed books
    print("Reviewed books with high ratings:", list(reviewed_books))

    return [(str(book[0]), float(book[1])) for book in reviewed_books]

In [ ]:
# Define the user you want recommendations for
user_id = 1

# Get genre-based recommendations
genre_recommendations = recommend_books_by_genre(user_id)
print("Recommendations based on preferred genres:", genre_recommendations)

# Get recommendations for highly-rated unread books
rating_recommendations = recommend_highly_rated_books(user_id, rating_threshold=4.0)
print("Recommendations based on high ratings:", rating_recommendations)

# Get recommendations based on other users' reviews
review_recommendations = recommend_books_based_on_reviews(user_id, review_threshold=4.0)
print("Recommendations based on other users' reviews:", review_recommendations)

Preferred genres: ['http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Genre1-1']
Recommendations based on preferred genres: [('http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Book1', 'http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Genre1-1'), ('http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Book4', 'http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Genre1-1'), ('http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Book5', 'http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Genre1-1')]
Recommendations based on high ratings: [('http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Book3', 4.0)]
Books that user has read: {'http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Book1', 'http://www.semanticweb.org/areej/ontologies/2024/10/Library-ontology-6#Book2'}
All books in the library: {'http://www.semant